In [15]:

import os 

import torch 
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms

from tqdm.auto import tqdm
import numpy as np
import cv2
import matplotlib.pyplot as plt

from nn_module import DML

In [17]:
down_width = 360  
down_height = 360 

data_X = []
data_Y = []
folder_names = os.listdir('test_data')

class_targets = dict(zip(folder_names, range(len(folder_names))))
 
for folder in folder_names:
    for img_name in os.listdir(f'test_data/{folder}'):
        img1 = cv2.imread(f'test_data/{folder}/{img_name}')
        img1 = torch.FloatTensor(cv2.resize(img1, (down_width, down_height), interpolation=cv2.INTER_LINEAR))
        data_X.append(img1.unsqueeze(0))
        data_Y.append(class_targets[folder])

data_X = torch.cat(data_X, axis=0)
data_X = data_X.permute(0, 3, 1, 2)
data_Y = torch.FloatTensor(data_Y)

data_size = data_X.shape[0]

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set trunk model and replace the softmax layer with an identity function
pretrain_resnet = torchvision.models.resnet18(pretrained=True)
input_embedding_size = pretrain_resnet.fc.out_features
pretrain_resnet = torch.nn.DataParallel(pretrain_resnet.to(device))

In [22]:
data_loader = DataLoader(data_X, batch_size=10, shuffle=False)

embeddings = torch.zeros((data_size, input_embedding_size))
left_pointer = 0
for batch in tqdm(data_loader):
    embeddings[left_pointer:left_pointer+batch.shape[0]] = pretrain_resnet(batch).cpu()
    left_pointer += batch.shape[0]

  0%|          | 0/30 [00:00<?, ?it/s]